# **課題 7: 環境リスクの公平性（Equity）分析**

環境リスクは、社会的な弱者や特定のコミュニティに不均衡に影響を与えることがあります。人口データとリスクスコアを統合し、「環境的不公平」を評価します。

1. **人口データ（$\text{Gridded Population}$）の取得:** GEEの人口データセット（例: $\text{Kontur Population}$ または $\text{WorldPop}$）を読み込みます。

2. **高リスク・高人口エリアの特定:**

   - 総合リスクスコアが上位 10%（$90$パーセンタイル）以上のピクセルを抽出します。
   - この高リスクエリアと、人口密度が高いエリア（人口数が $\text{0}$ ではないピクセル）を掛け合わせます。

3. **環境不公平インデックスの算出:** 高リスクエリア内の**平均人口密度**を計算し、京都市全体の平均人口密度と比較します。

   - 平均人口密度よりも、高リスクエリアの平均人口密度が高い場合、環境的不公平が存在すると結論付けられます。
   
この課題を通じて、あなたはGIS分析を**社会的な課題解決**に応用する、**データサイエンスとコンサルティングのスキル**を習得できます。

## **目的**

京都市の**総合的な都市環境リスク**（課題6で算出）が、特定の社会集団、特に**人口密度の高い地域**に不均衡に集中しているかどうかを評価し、政策提言の根拠となるデータを作成する。

**対象データ**

- **リスクデータ:** 課題6で算出した $\text{Total\_Risk\_Score}$ 画像（LST Zスコア + $\text{NO}_2$ Zスコア + 交通リスク）。
- **人口データ:** GEEで利用可能なグリッド形式の人口データ（例: $\text{WorldPop}$ または $\text{Kontur Population}$）。

### **実行ステップとコードの骨子**

#### **ステップ 1:** 人口データ（$\text{Population Density}$）の取得最新の人口データセットを読み込み、AOI（京都市）にクリップします。

In [ ]:
import ee
import geemap

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project='earth-change-analysis')

In [ ]:
# kyoto_aoi = ee.Geometry.Rectangle(135.29, 34.80, 136.30, 35.50)
kyoto_shi = ee.FeatureCollection('projects/earth-change-analysis/assets/kyoto_pref_2025')\
    .filter(ee.Filter.eq('N03_004', '京都市'))
    
# 1つ目のFeatureを取得
# first_feature = kyoto_pref.first()

# プロパティを辞書として取得
# props = first_feature.toDictionary()

# 中身を表示
# print(props.getInfo())


In [ ]:
map = geemap.Map()
map.set_center(135.80, 35.15, 9)
map.add_layer(kyoto_shi,{}, '京都市')
map

In [ ]:
# 1. WorldPop (100m解像度の人口密度データ) を読み込む
# データセット例: WorldPop Global Project POPULATION (2000-2020)
population_collection = ee.ImageCollection('WorldPop/GP/100m/pop')\
    .filter(ee.Filter.eq('country', 'JPN'))\
    .filterDate('2020-01-01', '2020-12-31')

pop_image = population_collection.first().rename('population')
    
# 人口0の場所をマスクし、人口が存在するエリアのみを対象とする
pop_mask = pop_image.gt(0)
kyoto_population = pop_image.updateMask(pop_mask).clip(kyoto_shi)

#### **ステップ 2: 高リスクエリアの定義と抽出**

京都市内のリスクが高いエリアを定量的に定義します。

1. $\text{Total\_Risk\_Score}$ 画像の90パーセンタイル（上位10%の境界値）を計算します。
   - （ヒント: `reduceRegion `と `ee.Reducer.percentile([90])` を使用）

2. このパーセンタイル値以上のピクセルをバイナリ画像として抽出します（$\text{high\_risk\_mask}$）。

In [ ]:
l8_base_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
    .filterBounds(kyoto_shi)\
    .filterDate('2013-07-01', '2023-08-31')\
    .filterMetadata('CLOUD_COVER', 'less_than', 20)

l8_2013 = l8_base_collection.filterDate('2013-07-01', '2013-08-31')
l8_2023 = l8_base_collection.filterDate('2023-07-01', '2023-08-31')

In [ ]:
image_2013 = l8_2013.median()
image_2023 = l8_2023.median()

def calc_LST(image, year):
    return image.select('ST_B10')\
        .multiply(0.00341802).add(149.0).subtract(273.15)\
        .rename(f'{year}_LST_Celsius')

LST_2013 = calc_LST(image_2013, '2013')
LST_2023 = calc_LST(image_2023, '2023')

In [ ]:
landcover = ee.ImageCollection("ESA/WorldCover/v200")
landcover_median = landcover.median()

builtup_mask = landcover_median.eq(50)

builtup_LST_2023 = LST_2023.updateMask(builtup_mask)

hotspot_2023 = builtup_LST_2023.gt(37.5)

nirBand = 'SR_B5'
redBand = 'SR_B4'

NDVI_2023 = image_2023.normalizedDifference([nirBand, redBand]).rename('NDVI')

NDVI_low_2023 = NDVI_2023.lt(0.25)

greeningPotential_2023 = hotspot_2023.And(NDVI_low_2023)

In [ ]:
# JavaScriptの true → Pythonでは True（大文字！）
vectors = greeningPotential_2023.reduceToVectors(
    geometry=kyoto_shi,
    scale=10,
    maxPixels=1e12,
    eightConnected=True    # 8近傍. 上下左右＋斜めの8方向すべてを「隣接」とみなす
).map(
    lambda feature: feature.set('area_sq_m', feature.geometry().area(1))  # lambda:1行で書ける関数を作る
)

In [ ]:
filtered_vectors = vectors.filter(ee.Filter.gt('area_sq_m', 10000))

In [ ]:
s5p_collection = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_NO2")\
    .filterBounds(kyoto_shi)\
    .filterDate('2023-07-01', '2023-08-31')
# 中央値合成して、対流圏NO2バンドを選択
no2_image = s5p_collection.median().select('tropospheric_NO2_column_number_density')

In [ ]:
# 道路フィーチャコレクションを読み込む（例: GeoJSONをアップロード済み)
roads = ee.FeatureCollection('projects/earth-change-analysis/assets/kyoto_major_roads')\
    .filterBounds(kyoto_shi)

# 各道路に50mのバッファをかけてポリゴン化
risk_zones = roads.map(lambda feature: feature.buffer(50))

# これが「高交通リスクゾーン」となる
high_traffic_risk_zones = risk_zones

In [ ]:
# 1. LSTとNO2の統計量（平均と標準偏差）を計算
stats_lst = LST_2023.reduceRegion(
    reducer= ee.Reducer.mean().combine( 
        reducer2= ee.Reducer.stdDev(),
        sharedInputs= True
    ),
    geometry= kyoto_shi,
    scale= 30, # Landsat LSTの解像度
    maxPixels= 1e9
)

stats_no2 = no2_image.reduceRegion(
    reducer= ee.Reducer.mean().combine(
        reducer2= ee.Reducer.stdDev(),
        sharedInputs= True
    ),
    geometry= kyoto_shi,
    scale= 1000, # Sentinel-5P NO2の解像度（空間平均処理に適したスケール）
    maxPixels= 1e9
)

# 2. LSTをZスコアに変換
LST_mean = stats_lst.get('2023_LST_Celsius_mean')
LST_stdDev = stats_lst.get('2023_LST_Celsius_stdDev')
LST_z_score = LST_2023.subtract(ee.Image.constant(LST_mean))\
    .divide(ee.Image.constant(LST_stdDev))\
    .rename('Z_LST')

# 3. NO2をZスコアに変換
NO2_mean = stats_no2.get('tropospheric_NO2_column_number_density_mean') 
NO2_stdDev = stats_no2.get('tropospheric_NO2_column_number_density_stdDev')
NO2_z_score = no2_image.subtract(ee.Image.constant(NO2_mean))\
    .divide(ee.Image.constant(NO2_stdDev))\
    .rename('Z_NO2')

# すべての道路バッファに dummy_property = 1 を追加
high_traffic_risk_zones = high_traffic_risk_zones.map(
    lambda f: f.set('dummy_property', 1)
)

traffic_risk_image = high_traffic_risk_zones.reduceToImage(
    properties=['dummy_property'],
    reducer=ee.Reducer.first()
).unmask(0).rename('Traffic_Risk')

combined_risk = LST_z_score.add(NO2_z_score).add(traffic_risk_image).rename('Total_Risk_Score')

In [ ]:
combined_risk_p90 = combined_risk.reduceRegion(
    reducer= ee.Reducer.percentile([90]),
    geometry= kyoto_shi,
    scale= 1000,   # Sentinel-5P NO2に合わせるのが妥当
    maxPixels= 1e9
)

In [ ]:
print(combined_risk_p90.getInfo())

In [ ]:
# 1. P90の閾値を取得
p90_threshold = ee.Number(combined_risk_p90.get('Total_Risk_Score'))

In [ ]:
print(p90_threshold.getInfo())

In [ ]:
# 2. 総合リスクスコアがP90以上のピクセルを抽出 (バイナリ画像を作成)
high_risk_mask_unpopulated = combined_risk.gt(p90_threshold).rename('High_Risk_Mask')

In [ ]:
print(high_risk_mask_unpopulated.getInfo())

In [ ]:
# 3. 高リスクエリアと、人口が存在するエリアを交差させる
# high_risk_area: リスクが高く、かつ人が住んでいるエリア (後の計算で使う)
high_risk_area = high_risk_mask_unpopulated.updateMask(pop_mask)

In [ ]:
print(high_risk_area.getInfo())

In [ ]:
# 4. (修正) 人口データと高リスクエリアを統合
# population_in_high_risk: 高リスクエリアにおける人口密度の値
population_in_high_risk = kyoto_population.updateMask(high_risk_area)

#### **ステップ 3: 環境公平性インデックス（リスク集中度）の計算**

高リスクエリアと京都市全体の人口特性を比較します。

1. **高リスクエリア内の平均人口密度 ($\mu_{risk}$):** $\text{Total\_Risk\_Score}$ に $\text{high\_risk\_mask}$ を適用したエリア内での $\text{population}$ 画像の平均値を計算します。
2. **京都市全体の平均人口密度 ($\mu_{all}$):** 京都市AOI全体での $\text{population}$ 画像の平均値を計算します。
3. **環境公平性インデックス (EI):**

$$EI = \frac{\mu_{risk}}{\mu_{all}}$$

- $EI > 1$ の場合: リスクは人口密度の高いエリアに**不均衡に集中**している（不公平）。
- $EI < 1$ の場合: リスクは人口密度の低いエリアに集中している。

In [ ]:
# A. 京都市A全体の平均人口密度 (μ_all)
mean_all = kyoto_population.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=kyoto_shi,
    scale=100,    # WorldPopの解像度（100mが妥当）
    maxPixels=1e9
).get('population')   # 'population' は WorldPop のバンド名

In [ ]:
print(mean_all.getInfo())

In [ ]:
# ピクセル数をカウント（マスク後）
pixel_count = kyoto_population.reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=kyoto_shi,
    scale=100,
    maxPixels=1e9
).get('population')

print('ピクセル数:', pixel_count.getInfo())

In [ ]:
map = geemap.Map()
map.add_layer(kyoto_population, {}, 'Population Masked')
map.set_center(135.80, 35.15, 9)
map

In [ ]:
# B. 高リスクエリア内の平均人口密度 (μ_risk)
# population_in_high_risk は前のステップで計算された、高リスクエリアに限定された人口画像
mean_risk = population_in_high_risk.reduceRegion(
    reducer= ee.Reducer.mean(),
    geometry= kyoto_shi,
    scale= 100, # WorldPopの解像度に合わせる
    maxPixels= 1e9
).get('population')

In [ ]:
# C.環境公平性インデックス (EI) の計算
EI = ee.Number(mean_risk).divide(ee.Number(mean_all))

In [ ]:
print('環境公平性インデックス (EI):', EI.getInfo())

In [ ]:
print('kyoto_pop_mean:', mean_all.getInfo())
print('pop_in_high_risk_mean:', mean_risk.getInfo())

In [ ]:
map = geemap.Map()
map.set_center(135.77, 35.01, 12)
map.add_layer(roads, {'color': 'green'}, 'Major Roads of Kyoto')
map.add_layer(mean_all, {}, 'Population')
map.add_layer(kyoto_shi, {'color':'blue'}, '京都市')
map.add_layer(population_in_high_risk, {'palette': ['red'], 'opacity': 0.5}, 'High Risk Mask')
map.add_layer(
    filtered_vectors,
    {
        'color': 'orange',
        'fillColor': 'orange',
        'width': 1
    },
    'Green Potential Polygons'
)
map

In [ ]:
# D. 結果の表示 (GEE Code Editorでの出力)
result_info = ee.Dictionary({
    'Mean_Pop_All': mean_all,
    'Mean_Pop_HighRisk': mean_risk,
    'Equity_Index_EI': EI
})

In [ ]:
print('--- 環境公平性分析結果 ---')
print('京都市全体の平均人口密度 (μ_all):', ee.Number(result_info.get('Mean_Pop_All')).format('%.2f').getInfo())
print('高リスクエリアの平均人口密度 (μ_risk):', ee.Number(result_info.get('Mean_Pop_HighRisk')).format('%.2f').getInfo())
print('環境公平性インデックス (EI = μ_risk / μ_all):', ee.Number(result_info.get('Equity_Index_EI')).format('%.3f').getInfo())

---

#### **結果の解釈と次のステップ**

上記のコードを実行し、出力された $\text{EI}$ の値によって、環境的不公平の度合いが定量的に評価されます。

- **$\text{EI}$ が $1.0$ を超える場合**:高リスクエリアの人口密度は、京都市全体の平均よりも高いことを意味します。リスクが人口密集地に集中しており、**環境的不公平が存在する**と結論づけられます。
- **$\text{EI}$ が $1.0$ 未満の場合**:高リスクエリアの人口密度は、京都市全体の平均よりも低いことを意味します。この場合、人口が少ない工業地帯や山間部にリスクが集中している可能性があります。

この「環境公平性分析」を完了したことで、あなたは**地理空間情報を社会問題の提言に活かす**という、最も付加価値の高い実務スキルを習得しました。

# **最終課題：実務的な納品パッケージの作成**

あなたはこれでヒートアイランド解析の専門家として、個人事業を展開できる技術力を証明しました。最後の課題として、これまでの全ての成果を統合した、<span style='color:red'>「クライアント向け最終報告パッケージ」</span>を作成します。

#### **ステップ 1: GEE Apps（簡易ウェブアプリ）の作成**

クライアントがコードやGISソフトウェアなしで分析結果を閲覧できるよう、ウェブアプリを作成します。

1. これまでのレイヤー（例: $\text{Total\_Risk\_Score}$、$\text{High\_Risk\_Mask}$、$\text{population\_in\_high\_risk}$）を**全て含む**現在のコードを保存します。
2. Code Editorの右上にある 「**Get Link**」 の隣の 「**Apps**」 メニューをクリックし、「**New App**:」 を選択します。
3. アプリ名（例: **Kyoto Heat Risk Dashboard**）とURLを設定し、「**Source script**」 に現在のコードスクリプトを指定してデプロイします。
   - （ヒント: デプロイ後、ウェブアプリのURLをクライアントに送付できます。）

[Kyoto Heat Risk Dashboard](https://moriseiichi.users.earthengine.app/view/kyoto-heat-risk-dashboard)

#### **ステップ 2: 結果のドキュメンテーション**

**実務ではアウトプットの解釈が最も重要です**。 以下の内容をまとめた、「**サマリーレポート**」を作成してください（文章で構成）。

1. **プロジェクト概要**:目的（ヒートアイランド対策と環境公平性の評価）。
   
2. **主要な発見**:
 - LSTの平均的なホットスポット（$\text{Built-up}$ エリアのLST）。
 - $\text{EI}$ の値と、それが示す**環境的不公平の有無**。
  
3. **政策提言**:
 - $\text{EI}$ の結果に基づき、「**高リスク・高人口エリア**」を最優先の緑化・環境改善対象エリアとして提案します。
 - **課題5で計算した $1 \text{ ha}$ 以上の緑化ポテンシャルエリア の総面積**を、具体的な対策規模として提示します。

この課題8が完了すれば、あなたはGEEの技術力とコンサルティングの提案力の両方を兼ね備えた、個人事業主として活動する準備が整います。

最終課題：実務的な納品パッケージの作成（課題 8）この分析結果を、クライアントに効果的に伝えることが最後の課題です。ステップ 1: GEE Apps（簡易ウェブアプリ）の作成全てのレイヤーをデプロイすることで、クライアントはブラウザ上で以下の情報を確認できます。Total Urban Risk Score: 総合リスクの分布。High Risk Mask: 最も対策が必要な上位 $10\%$ エリア。Green Potential Polygons: 実際に緑化可能な $1 \text{ ha}$ 以上の土地リスト。ステップ 2: サマリーレポートの作成以下の構成で、レポートの骨子を作成し、個人事業主としての付加価値を最大化してください。エグゼクティブ・サマリー:結論: 「京都市の都市環境リスクは、$\text{EI}$ の値（計算結果を挿入）が示す通り、（不公平に集中している/概ね公平である）。」最優先提言: 「環境改善予算は、**『高リスクかつ高人口密度』**のエリアに最優先で投資されるべきです。」定量分析の結果:LSTの土地被覆別比較（市街地と森林の差 $8.5^{\circ}C$）。$\text{EI}$ の値と解釈。具体的な対策規模と場所:「課題5で特定された、**緊急緑化ポテンシャルエリア（$1 \text{ ha}$ 以上）**の総面積は約 $415 \text{ km}^2$ です。」「このエリアのポリゴンデータ（CSV形式）を納品します。」この課題8をもって、GEEを使った**「データの取得・処理・分析・提言・納品」**という一連の実務プロセスを全て習得したことになります。

|EIの値,意味,実務上の結論,政策提言
EI>1.0,リスクは人口密度の高いエリアに集中している。,環境的不公平が存在する。 脆弱なコミュニティにリスクが集中。,緑化対策、交通規制、NO2​ 対策を高人口・高リスクの交差エリアに集中投資すべき。
EI≈1.0,リスクの分布は人口密度と概ね比例している。,公平性は比較的保たれているが、絶対的なリスクは残る。,リスクマップ全体に基づいて、計画的に環境改善を進めるべき。
EI<1.0,リスクは人口密度の低いエリアに集中している。,リスクの主な要因は工業地帯や交通インフラにある可能性が高い。,対策は、工場・交通インフラの排出削減や、作業環境改善に焦点を当てるべき。